In [7]:
import geemap
import time
import os
import time
import ee
import os
import numpy
import pandas

In [876]:
ee.Authenticate()


Successfully saved authorization token.


In [9]:
ee.Initialize()

In [47]:
#This feature collection is a sample (n=5) of lakes/ponds from nhd waterbodies within the Ohio River Basin
points = ee.FeatureCollection('projects/ee-samsillen0/assets/OhioRiver_nhd_sample')


In [48]:
#LS Collection 2 has a different scaling parameter that needs to be applied to the optical bands

def scale(image):

#Apply the scaling factors to the appropriate bands.
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

#Replace the original bands with the scaled ones
  return image.addBands(opticalBands , overwrite = True) #overwrite = True replaces the original optical bands with the scaled bands

band_names = ['Blue', 'Green', 'Red','Nir', 'Swir1', 'Swir2', 'qa']

ls5_bands = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7', 'QA_PIXEL']

ls7_bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']

ls8_bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'] # SR_B1 == ultra blue

ls5_data = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
    .map(scale)\
    .select(ls5_bands, band_names)\
    
#Landsat 7 scan line error problems addressed by filtering around it
ls7_data = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")\
     .map(scale)\
     .filterDate('1999-04-15' , '2003-05-30')\
     .select(ls7_bands, band_names)\

ls8_data = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
    .map(scale)\
    .select(ls8_bands,band_names)\

ls2 = ls5_data.merge(ls7_data).merge(ls8_data)\
  .filter(ee.Filter.lt('CLOUD_COVER', 50))

In [49]:

#Now we will create masks for our image. Masks are binary images used to remove unwanted pixels from the final layer. 
# For example, we could create a cloud mask to remove cloudy pixels, a snow/ice mask to remove snow or ice pixels, a water mask to remove non-water pixels, etc. 

#Below are the updated bit numbers within the 'QA_PIXEL' band for LS Collection 2 data. 

#    Bit 0: Fill
#    Bit 1: Dilated Cloud
#    Bit 2: Cirrus (high confidence)
#    Bit 3: Cloud
#    Bit 4: Cloud Shadow
#    Bit 5: Snow
#    Bit 6: Clear
#        0: Cloud or Dilated Cloud bits are set
#        1: Cloud and Dilated Cloud bits are not set
#    Bit 7: Water
#    Bits 8-9: Cloud Confidence
#        0: None
#        1: Low
#        2: Medium
#        3: High
#    Bits 10-11: Cloud Shadow Confidence
#        0: None
#        1: Low
#        2: Medium
#        3: High
#    Bits 12-13: Snow/Ice Confidence
#        0: None
#        1: Low
#        2: Medium
#        3: High
#    Bits 14-15: Cirrus Confidence
#        0: None
#        1: Low
#        2: Medium
#        3: High

In [50]:
#The following functions 'unpack' the information stored within the QA band. For reference, the Water bit, bit 7, would have a starting bit of 7 and a bit width of 1

def Unpack(bitBand, startingBit, bitWidth):
  return (ee.Image(bitBand)\
  .rightShift(startingBit)\
  .bitwiseAnd(ee.Number(2).pow(ee.Number(bitWidth)).subtract(ee.Number(1)).int()))


def UnpackAll(bitBand, bitInfo):
  unpackedImage = ee.Image.cat([Unpack(bitBand, bitInfo[key][0], bitInfo[key][1]).rename([key]) for key in bitInfo])
  return unpackedImage

def clipImage(image):
  return image.clip(lake.geometry())
  
def masking(image):
    bitAffected = {
    'Water': [7,1], #starting bit followed by bit width
    'Cloud': [3, 1],
    'CloudShadow': [4, 1],
    'SnowIceConfidence': [5, 1]  
    }

    qa = image.select('qa')
  
    qaUnpack = UnpackAll(qa, bitAffected)
  
    fWater = qaUnpack.select('Water').eq(1)
  
    cScore = qaUnpack.select('Cloud').eq(1)\
    .Or(qaUnpack.select('CloudShadow').eq(1))\
    .Or(qaUnpack.select('SnowIceConfidence').eq(1))\
  

    cScore = cScore.reduceRegion(ee.Reducer.sum(),lake.geometry(), 30).get('Cloud')
    
  #Update mask on imagery and add Pekel occurrence band for data export.
    masked = image.addBands(pekel.select('occurrence'))\
    .updateMask(fWater).set({'cScore':cScore})
  
    return masked

In [51]:
def AddFmask(image):
    bitInfo = {
    'Cloud': [3, 1],
    'CloudShadow': [4, 1], 
    'SnowIce': [5, 1],
    'Water': [7, 1]
    }
    
    temp = UnpackAll(image.select(['qa']), bitInfo)
    
    fmask = (temp.select(['Water']).rename(['fmask'])
    .where(temp.select(['SnowIce']), ee.Image(4)) #4 because we're taking SnowIce bit number (5) and subtracting 1
    .where(temp.select(['CloudShadow']), ee.Image(3))
    .where(temp.select(['Cloud']), ee.Image(2))
    .mask(temp.select(['Cloud']).gte(0))) 
    #mask the fmask so that it has the same footprint as the quality (BQA) band
    return(image.addBands(fmask))

In [52]:
#These functions are for calculating DSWE. Thresholds (t1,t2, etc) need to be updated for collection 2 data, I believe. 

def Mndwi(image):
  return(image.normalizedDifference(['Green', 'Swir1']).rename('mndwi'))

def Mbsrv(image):
  return(image.select(['Green']).add(image.select(['Red'])).rename('mbsrv'))

def Mbsrn(image):
  return(image.select(['Nir']).add(image.select(['Swir1'])).rename('mbsrn'))

def Ndvi(image):
  return(image.normalizedDifference(['Nir', 'Red']).rename('ndvi'))

def Awesh(image):
  return(image
  .expression('Blue + 2.5 * Green + (-1.5) * mbsrn + (-0.25) * Swir2', {
    'Blue': image.select(['Blue']),
    'Green': image.select(['Green']),
    'mbsrn': Mbsrn(image).select(['mbsrn']),
    'Swir2': image.select(['Swir2'])
  }))

def Dswe(i):
   mndwi = Mndwi(i)
   mbsrv = Mbsrv(i)
   mbsrn = Mbsrn(i)
   awesh = Awesh(i)
   swir1 = i.select(['Swir1'])
   nir = i.select(['Nir'])
   ndvi = Ndvi(i)
   blue = i.select(['Blue'])
   swir2 = i.select(['Swir2'])
  
   t1 = mndwi.gt(0.124)
   t2 = mbsrv.gt(mbsrn)
   t3 = awesh.gt(0)
   t4 = (mndwi.gt(-0.44)
   .And(swir1.lt(900))
   .And(nir.lt(1500))
   .And(ndvi.lt(0.7)))
   t5 = (mndwi.gt(-0.5)
   .And(blue.lt(1000))
   .And(swir1.lt(3000))
   .And(swir2.lt(1000))
   .And(nir.lt(2500)))
  
   t = t1.add(t2.multiply(10)).add(t3.multiply(100)).add(t4.multiply(1000)).add(t5.multiply(10000));
  
   noWater = (t.eq(0)
   .Or(t.eq(1))
   .Or(t.eq(10))
   .Or(t.eq(100))
   .Or(t.eq(1000)))
   hWater = (t.eq(1111)
   .Or(t.eq(10111))
   .Or(t.eq(11011))
   .Or(t.eq(11101))
   .Or(t.eq(11110))
   .Or(t.eq(11111)))
   mWater = (t.eq(111)
   .Or(t.eq(1011))
   .Or(t.eq(1101))
   .Or(t.eq(1110))
   .Or(t.eq(10011))
   .Or(t.eq(10101))
   .Or(t.eq(10110))
   .Or(t.eq(11001))
   .Or(t.eq(11010))
   .Or(t.eq(11100)))
   pWetland = t.eq(11000)
   lWater = (t.eq(11)
   .Or(t.eq(101))
   .Or(t.eq(110))
   .Or(t.eq(1001))
   .Or(t.eq(1010))
   .Or(t.eq(1100))
   .Or(t.eq(10000))
   .Or(t.eq(10001))
   .Or(t.eq(10010))
   .Or(t.eq(10100)))
  
   iDswe = (noWater.multiply(0)
   .add(hWater.multiply(1))
   .add(mWater.multiply(2))
   .add(pWetland.multiply(3))
   .add(lWater.multiply(4)))
  
   return iDswe.rename('dswe')

In [53]:
# Calculuate hillshades/hillshadows to correct DWSE
# For collection one, azimuth field was 'SOLAR_AZIMUTH', I believe. It was changed to 'SUN_AZIMUTH'. 
# Also, there is no zenith field stored within landsat collection 2. You have to take the 'SUN_ELEVATION' field and do 90 - 'SUN_ELEVATION' for zenith. 

def CalcHillShades(image, geo):

  MergedDEM = ee.Image("users/eeProject/MERIT").clip(geo.buffer(30))
  
  hillShade = (ee.Terrain.hillshade(MergedDEM, ee.Number(image.get('SUN_AZIMUTH')),
  image.get('SUN_AZIMUTH')).rename(['hillShade']))
  return hillShade
  
def CalcHillShadows(image, geo):
  MergedDEM = ee.Image("users/eeProject/MERIT").clip(geo.buffer(30))
  hillShadow = (ee.Terrain.hillShadow(MergedDEM, ee.Number(image.get('SUN_AZIMUTH')),
  ee.Number(90).subtract(image.get('SUN_ELEVATION')), 30).rename(['hillShadow']))
  return hillShadow

In [54]:
def pull(image):

  def waterOnly(image):
    d = Dswe(image).select('dswe')
    f = AddFmask(image).select('fmask')
    cScore = f.eq(2).Or(f.eq(3)).Or(f.eq(4))
    cScore = cScore.reduceRegion(ee.Reducer.sum(),lake.geometry(),30).get('fmask')
    d = Dswe(image).select('dswe')
    h = CalcHillShades(image, lake.geometry()).select('hillShade')
    hs = CalcHillShadows(image, lake.geometry()).select('hillShadow')
   
    return image.addBands(f).addBands(d).addBands(h).addBands(hs).updateMask(f.lte(2)).updateMask(d.eq(1).Or(d.eq(2))).set({'cScore':cScore})
   
  waterOut = waterOnly(image)
  waterMask = waterOut.select('dswe').eq(1)
  waterOut = waterOut.updateMask(waterMask)
  sampledData = waterOut.reduceRegion(ee.Reducer.median(), lake.geometry(), 10)  
  
  output = shell.set({"blue": sampledData.get('Blue')})\
  .set({'date': ee.Date(image.get('system:time_start'))})\
  .set({"blue": sampledData.get('Blue')})\
  .set({"green": sampledData.get('Green')})\
  .set({"red": sampledData.get('Red')})\
  .set({"nir": sampledData.get('Nir')})\
  .set({"swir1": sampledData.get('Swir1')})\
  .set({"swir2": sampledData.get('Swir2')})\
  .set({"qa_pix": sampledData.get('qa')})\
  .set({'hillshade':sampledData.get('hillShade')})\
  .set({'hillshadow':sampledData.get('hillShadow')})\
  .set({'azimuth': image.get('SUN_AZIMUTH')})\
  .set({'sun_elev': image.get('SUN_ELEVATION')})\
  .set({'cScore': waterOut.get('cScore')})\
  .set({'Cloud_Cover': image.get('CLOUD_COVER')})\
  .set({'dswe': sampledData.get('dswe')})\
  .set({"pixelCount": waterOut.reduceRegion(ee.Reducer.count(), lake.geometry(), 30).get('Blue')})\


  return(output)


In [55]:
lakesort = pointsBuffer.sort('comid')

lakeID = lakesort.aggregate_array('comid').getInfo() 

dlDir = 'G:/My Drive/test_export' 
filesDown = os.listdir(dlDir)  

filesDown = [int(i.replace(".csv", "")) for i in filesDown]

lakeID  = [i for i in lakeID if i not in filesDown]

print(lakeID)


[908305, 4739613, 5212451, 5215825, 6884994]


In [56]:

for x in range(0,len(lakeID)):

  lake = ee.Feature(points.filter(ee.Filter.eq('comid', lakeID[x])).first())
  
  lake = lake.buffer(300)

  shell = ee.Feature(None)
  #FilterBounds for lake, update masks for water occurence, clouds, roads, etc.
  #Remove any images with clouds directly over the waterbody
  lsover = ls2.filterBounds(lake.geometry())\
  .map(clipImage)

  data = lsover.map(pull)
    
  dataOut = ee.batch.Export.table.toDrive(collection = data, \
                                            description = str(lakeID[x]),\
                                            folder = 'test_export',\
                                            fileFormat = 'csv')

  print(lakeID[x])


  dataOut.start()

908305
4739613
5212451
5215825
6884994
